In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Reload all modules imported with %aimport
%load_ext autoreload
%autoreload 1

In [2]:
import helper
%aimport helper

helper = helper.Helper()

In [3]:
import tensorflow as tf
import numpy as np
import os

## Load the data

In [4]:
data_dir = np.load('./data/train_test_data.npz')
X_train = data_dir['X_train']
X_test = data_dir['X_test']
y_train = data_dir['y_train']
y_test = data_dir['y_test']
data = np.concatenate((X_train, X_test))
labels = np.concatenate((y_train, y_test))

## Load models and histories

In [5]:
model_name0 = "Head only"
model0 = helper.loadModel(model_name0)
score0 = model0.evaluate(X_test, y_test, verbose=0)
history0 = helper.loadHistory(model_name0)

model_name1 = "Dense + Head"
model1 = helper.loadModel(model_name1)
score1 = model1.evaluate(X_test, y_test, verbose=0)
history1 = helper.loadHistory(model_name1)

### 1. Model structure test

In [7]:
# Set Classifier activation and loss depending on shape of target
if helper.y_OHE(y_train):
    num_cases = 2
    activation = "softmax"
    loss = 'categorical_crossentropy'
else:
    num_cases = 1
    activation = "sigmoid"
    loss = 'binary_crossentropy'
    

## Model0 Head Only
assert len(model0.layers) <= 3

n_samples, width, height = data.shape
dense_layer = model0.get_layer(name='dense_head')
# Dense layer tests
assert dense_layer.input.shape[1:] == tf.TensorShape([ np.prod(data.shape[1:]) ] )
assert dense_layer.output.shape[1:] == tf.TensorShape([num_cases])
### END HIDDEN TESTS

In [9]:
### BEGIN HIDDEN TESTS
num_features_1 = 64

dense_1 = model1.get_layer(name='dense_1')
dense_head = model1.get_layer(name='dense_head')

# Dense 1
assert dense_1.input.shape[1:] == tf.TensorShape([np.prod(data.shape[1:])])
assert dense_1.output.shape[1:] == tf.TensorShape([num_features_1])

# Dense head
assert dense_head.output.shape[1:] == tf.TensorShape([num_cases])

### END HIDDEN TESTS

### 2. Training result test

In [12]:
### BEGIN HIDDEN TESTS

# Create the key with which to look up accuracy, val_accuracy.  It depends on how the model was compiled.
acc_string = helper.acc_key(model=model1)
acc_metric, val_acc_metric = acc_string, "val_" + acc_string

assert history0[acc_string][-1] > 0.6
assert history0[val_acc_metric][-1] > 0.6
### END HIDDEN TESTS

In [13]:
### BEGIN HIDDEN TESTS
assert history1[acc_string][-1] > 0.75
assert history1[val_acc_metric][-1] > 0.75
### END HIDDEN TESTS

In [14]:
### BEGIN HIDDEN TESTS
assert history1[acc_string][-1] > history0[acc_string][-1]
assert history1[val_acc_metric][-1] > history0[val_acc_metric][-1]
### END HIDDEN TESTS

### 3. Model parameters number test

In [16]:
### BEGIN HIDDEN TESTS
assert model0.count_params() == (np.prod(data.shape[1:]) * num_cases + num_cases)
### END HIDDEN TESTS

In [17]:
### BEGIN HIDDEN TESTS
dense_layers = [layer for layer in model1.layers if 'dense' in layer.name]
num_new_parameters = (np.prod(data.shape[1:]) + 1) * num_features_1 + (num_features_1 + 1) * num_cases

if len(dense_layers) == 2:
    assert model1.count_params() == num_new_parameters
elif len(dense_layers) > 2:
    assert model1.count_params() > num_new_parameters
### END HIDDEN TESTS

### 4. Evaluate the model

In [18]:
### BEGIN HIDDEN TESTS
assert score0[1] > 0.6
### END HIDDEN TESTS

In [19]:
### BEGIN HIDDEN TESTS
assert score1[1] > 0.8
### END HIDDEN TESTS

In [20]:
### BEGIN HIDDEN TESTS
assert score1[1] >= score0[1]
### END HIDDEN TESTS